In [0]:
%load_ext autoreload
%autoreload 1
%aimport data.adress

In [0]:
import sys
sys.path.append("..")
import pickle
from pprint import pprint

### Validation Data

In [0]:
from data.adress import load_CHAT_transcripts

data = load_CHAT_transcripts()
data = data[["Speaker", "Transcript", "Transcript_clean", "Utterance", "Repetitive speech"]]
data.head()

## LLM-Based Detector

In [0]:
from openai import OpenAI

DATABRICKS_TOKEN    = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
DB_ENDPOINT_URL     = "https://adb-2035410508966251.11.azuredatabricks.net/serving-endpoints"

client = OpenAI(api_key=DATABRICKS_TOKEN, base_url=DB_ENDPOINT_URL)

In [0]:
def run(prompt, model, utt_per_query):
    outputs = []
    for (trn_tst, pt_id), transcript in data.groupby(level=["train_test", "patient_id"]):
        for end in range(transcript.shape[0]):
            # Skip provider turns
            if transcript.loc[(trn_tst, pt_id, end), "Speaker"] == "Provider":
                continue

            # Sliding window
            start = max(0, end - utt_per_query + 1)
            text = "\n".join(transcript.loc[(slice(None), slice(None), slice(start, end)), "Utterance"].to_list())

            # Execute query
            response = client.chat.completions.create(
                model=model, 
                messages=[
                    {
                        "role": "user", 
                        "content": prompt.format(text)
                    }
                ]
            )
            outputs.append(response.choices[0].message.content)

    return outputs

### Prompts
Best version: v1

In [0]:
prompt_v1 = ""

In [0]:
outputs = run(prompt_v1, "openai_gpt_4o", 1)

In [0]:
with open("outputs_repetitive_speech.pkl", "wb") as f:
    pickle.dump(outputs, f)